<a href="https://colab.research.google.com/github/malinphy/Embedding_calls/blob/main/BERT_MLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd
import random 
import re
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.preprocessing.sequence import pad_sequences


from tensorflow.keras.layers import Dense, Embedding, Flatten ,MultiHeadAttention,Dropout
from tensorflow.keras import Sequential

import matplotlib.pyplot as plt

In [ ]:
## 1) Sentences will be tokenized using the TextVectorization 
## 2) 12.5 % of the tokens will be replaced with the [MASK] token. Substituded tokens will be 
## chosen randomly. To select the random tokens, random sampling will be chosen 

## 3) segment id will be determined at the second step
## 4) positional embeddings will be directly taken from the article

In [ ]:
padding_length =  12  ## 128
# corpus_length = 128   ## 30000
embedding_dimension = 128   ## 128
vocab_size = 128  ## 30000
number_heads = 8



In [ ]:
trial_sentences = [
    'my name is john',
    'london is the capital is the england',
    'weather is cold at the northern hemisphere of the world',
    'today, i will go the school',
    # 'London is the at northern side of the world'
]


# prediction_sentence = 'my name is [MASK]'
# prediction_tokens = tokenizing_procedure(prediction_sentence, tf_tokenizer)


longest_sentence = []
for i in trial_sentences:
    longest_sentence.append(len(i.split()))
    
print(np.max(longest_sentence))

longest_sentence = np.max(longest_sentence)

10


In [ ]:
tf_tokenizer= Tokenizer()

def tokenizing_procedure(text,tokenizer,padding_length): ##### Return the corpus and the padded_sequences
  # this function works as I expected
  tf_tokenizer = tokenizer
  tf_tokens = tf_tokenizer.fit_on_texts(text)

  corpus = tf_tokenizer.word_index
  sequences = tf_tokenizer.texts_to_sequences(text)
  padded_sequences = pad_sequences(sequences,maxlen = padding_length)
  corpus['MASK'] = len(corpus)+1

  return padded_sequences, corpus


unmasked_sequence, corpus = tokenizing_procedure(trial_sentences, tf_tokenizer,padding_length)

print(unmasked_sequence)




[[ 0  0  0  0  0  0  0  0  3  4  2  5]
 [ 0  0  0  0  0  6  2  1  7  2  1  8]
 [ 0  0  9  2 10 11  1 12 13 14  1 15]
 [ 0  0  0  0  0  0 16 17 18 19  1 20]]


In [ ]:
def masking(corpus, padded_sequences,tokenizer):
  masked_corpus = corpus.copy()
  masked_corpus['[MASK]'] = len(corpus)+1
  corpus_tokens = []   #### TOKEN SET WITHOUT THE SPECIAL TOKENS
  for i,j in enumerate(masked_corpus):
      corpus_tokens.append(j)

  subs_tokens=random.sample(corpus_tokens,int(len(corpus_tokens)*0.12))
  subs_tokens

  subs_tokens_ids = [ ]  ## ID OF THE SUBS TOKENS

  for i in subs_tokens:
      subs_tokens_ids.append(tf_tokenizer.word_index[i])

  mask_token_id = 20
  masked_token_id = len(corpus)+1

  for i in range(len(corpus_tokens)):
        if i == (subs_tokens_ids[0]-1   ):
            corpus_tokens[i] = '[MASK]'
    
        if i == (subs_tokens_ids[1]-1):
            corpus_tokens[i] = '[MASK]'

  for i in range(padded_sequences.shape[0]):
        for j in range(padded_sequences.shape[1]):
          for k in range(len(subs_tokens_ids)):
            if padded_sequences[i][j] == (subs_tokens_ids[k]):

                padded_sequences[i][j] = masked_token_id 
            if padded_sequences[i][j] == (subs_tokens_ids[k]):
                
                padded_sequences[i][j] = masked_token_id 


  return padded_sequences, masked_token_id
        
masked_padded_sequence, masked_token_id = masking(corpus, unmasked_sequence, tf_tokenizer)
print('masked sequence','\n',masked_padded_sequence)
print('unmasked seqeunce','\n',unmasked_sequence)
print(masked_token_id)




masked sequence 
 [[ 0  0  0  0  0  0  0  0  3  4  2  5]
 [ 0  0  0  0  0  6  2  1  7  2  1  8]
 [ 0  0 22  2 10 11  1 12 13 14  1 15]
 [ 0  0  0  0  0  0 16 17 18 19  1 22]]
unmasked seqeunce 
 [[ 0  0  0  0  0  0  0  0  3  4  2  5]
 [ 0  0  0  0  0  6  2  1  7  2  1  8]
 [ 0  0 22  2 10 11  1 12 13 14  1 15]
 [ 0  0  0  0  0  0 16 17 18 19  1 22]]
22


In [ ]:
def segment_tokenizer(masked_padded_sequence):
  segment_ids = masked_padded_sequence.copy()
  for i in range(segment_ids.shape[0]):
    for j in range(segment_ids.shape[1]):
      if segment_ids[i][j] != 0:
        segment_ids[i][j] = 1

  return segment_ids

segment_ids = segment_tokenizer(masked_padded_sequence)
# def masked_text_maker(masked_padded_sequences, masked_token_id):

#   for i in range(masked_padded_sequences.shape[0]):
#         for j in range(masked_padded_sequences.shape[1]):
#             if masked_padded_sequences[i][j] != 0:
#                 x = (masked_padded_sequences[i][j]  )
#                 maskeli.append("".join(extended_tokens[x-1]))
                
                
#   maskeli_2 = []
#   for i in (masked_padded_sequences):
#       maskeli_2.append('\n')
#       for j in range(len(i)):
        
#           if (i[j] !=0):
# #            print(i[j])
#               maskeli_2.append("".join(extended_tokens[i[j] -1]))
    
    
#   y= " ".join(maskeli_2)

#   y = y.split('\n')
#   y = np.delete(y,0,0)
  
#   return y

# masked_text_maker(masked_padded_sequence, masked_token_id)

In [ ]:
total_tokens = segment_ids + masked_padded_sequence

total_tokens.shape
print(unmasked_sequence)

[[ 0  0  0  0  0  0  0  0  3  4  2  5]
 [ 0  0  0  0  0  6  2  1  7  2  1  8]
 [ 0  0 22  2 10 11  1 12 13 14  1 15]
 [ 0  0  0  0  0  0 16 17 18 19  1 22]]


In [ ]:
# def masking_procedure (text,tok):
    
#     # tf_tokenizer = Tokenizer()
#     tf_tokenizer = tok
#     tf_tokens=tf_tokenizer.fit_on_texts(text)

#     corpus = tf_tokenizer.word_index  ### TOKEN SET WITH TOKEN IDS WITHOUT THE SPECIAL TOKENS 
#     extended_corpus = corpus.copy()   ### TOKEN SET WITH SPECIAL TOKENS

#     extended_corpus['MASK'] = len(corpus)+1

#     corpus_tokens = []   #### TOKEN SET WITHOUT THE SPECIAL TOKENS
#     for i,j in enumerate(corpus):
#         corpus_tokens.append(j)
# #         print(i,j)

        
#     extended_tokens = []
#     for i,j in  enumerate(extended_corpus):
#     #     print(i,j)
#         extended_tokens.append(j)
#     # print('maskelenmis_corpus')
#     # print('-------')
#     subs_tokens=random.sample(corpus_tokens,int(len(corpus_tokens)*0.12)) ## RANDOMLY SELECTED TOKENS TO BE MASKED
# #     print(subs_tokens)

#     sequences = tf_tokenizer.texts_to_sequences(text)
#     padded_sequences= pad_sequences(sequences)
#     padded_sequences

#     substitute_padded_sequences= padded_sequences.copy()
#     actual_padded_sequences = padded_sequences.copy()

#     subs_tokens_ids = [ ]  ## ID OF THE SUBS TOKENS

#     for i in subs_tokens:
#         subs_tokens_ids.append(tf_tokenizer.word_index[i])
    
    
#     mask_token_id = 20
# #     print(subs_tokens_ids)



#     for i in range(len(extended_tokens)):
#         if i == (subs_tokens_ids[0]-1   ):
#             extended_tokens[i] = '[MASK]'
    
#         if i == (subs_tokens_ids[1]-1):
#             extended_tokens[i] = '[MASK]'
        
        
        
#     maskeli = []
#     for i in range(substitute_padded_sequences.shape[0]):
#         for j in range(substitute_padded_sequences.shape[1]):
#             if substitute_padded_sequences[i][j] != 0:
#                 x = (substitute_padded_sequences[i][j]  )
#                 maskeli.append("".join(extended_tokens[x-1]))
                
                
#     maskeli_2 = []
#     for i in (substitute_padded_sequences):
#         maskeli_2.append('\n')
#         for j in range(len(i)):
        
#             if (i[j] !=0):
# #             print(i[j])
#                 maskeli_2.append("".join(extended_tokens[i[j] -1]))
    
    
#     y= " ".join(maskeli_2)

#     y = y.split('\n')
#     y = np.delete(y,0,0)
#     y
     
#     segment_ids = substitute_padded_sequences.copy()

#     for i in range(segment_ids.shape[0]):
#         for j in range(segment_ids.shape[1]):
#             if (segment_ids[i][j] != 0):
#                 segment_ids[i][j] = 1
#     segment_ids = np.array(segment_ids)
    
#     return (y , extended_tokens, segment_ids,substitute_padded_sequences,actual_padded_sequences)

In [ ]:
MAX_LEN = 256
EMBED_DIM = 128



def get_pos_encoding_matrix(max_len, d_emb):
    pos_enc = np.array(
        [
            [pos / np.power(10000, 2 * (j // 2) / d_emb) for j in range(d_emb)]
            if pos != 0
            else np.zeros(d_emb)
            for pos in range(max_len)
        ]
    )
    pos_enc[1:, 0::2] = np.sin(pos_enc[1:, 0::2])  # dim 2i
    pos_enc[1:, 1::2] = np.cos(pos_enc[1:, 1::2])  # dim 2i+1
    return pos_enc


positional_embeddings = get_pos_encoding_matrix(MAX_LEN, EMBED_DIM)

In [ ]:
# pos_enc = get_pos_encoding_matrix(MAX_LEN, EMBED_DIM)
# # pos_enc + segment_ids

# total_encoding = masked_sequence+segment_ids # +pos_enc  (bi sekilde positional encodingi sokmam gerek )

In [ ]:
total_tokens #  INPUT SEQUENCE 
unmasked_sequence  # LOGITS
total_tokens.shape

(4, 12)

In [ ]:
## bu kismi embedding layer'i ekleyecegim 
def vectorizer(X,vocab_size, embedding_dimension):
  embedding_out =  layers.Embedding(vocab_size, embedding_dimension)(X)

  return (embedding_out)

x_input_vectors = vectorizer(total_tokens,1000,embedding_dimension) #### MLM INPUT X
print('total_vectors_shape', x_input_vectors.shape)
y_input_vectors = vectorizer(unmasked_sequence, 1000,embedding_dimension) #### MLM INPUT Y
print('total_sequences_shape', y_input_vectors.shape)

total_vectors_shape (4, 12, 128)
total_sequences_shape (4, 12, 128)


In [ ]:
y = layers.Embedding(
        vocab_size, embedding_dimension, name="word_embedding"
    )(unmasked_sequence)

y.shape

unmasked_sequence.shape

(4, 12)

In [ ]:
inputs = layers.Input((12), dtype=tf.int64)

word_embeddings = layers.Embedding(
        vocab_size, embedding_dimension, name="word_embedding"
    )(inputs)
# position_embeddings = layers.Embedding(
#         input_dim=padding_length,
#         output_dim=config.EMBED_DIM,
#         # weights=[get_pos_encoding_matrix(padding_length, config.EMBED_DIM)],
#         name="position_embedding",
#     )(tf.range(start=0, limit=config.MAX_LEN, delta=1))
embeddings = word_embeddings# + position_embeddings

encoder_output = embeddings
    
for i in range(1):
    attention_output = layers.MultiHeadAttention(
    num_heads=8,
    key_dim=embedding_dimension // 8)(encoder_output, encoder_output, encoder_output)
    attention_output = layers.Dropout(0.1)(attention_output)
    attention_output = layers.LayerNormalization(
    epsilon=1e-6)(encoder_output + attention_output)

    # Feed-forward layer
ffn = keras.Sequential(
        [
            layers.Dense(128, activation="relu"),
            layers.Dense(embedding_dimension),
        ]
        )
ffn_output = ffn(attention_output)
ffn_output = layers.Dropout(0.1)(ffn_output)
sequence_output = layers.LayerNormalization(
        epsilon=1e-6)(attention_output + ffn_output)
        # encoder_output = bert_module(encoder_output, encoder_output, encoder_output, i)

mlm_output = layers.Dense(vocab_size,  activation="softmax")(sequence_output)
mlm_model = keras.Model(inputs, mlm_output, name="masked_bert_model")
mlm_model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'Adam'
)
# optimizer = keras.optimizers.Adam(learning_rate=config.LR)
# mlm_model.compile(optimizer=optimizer)
    # return mlm_model
mlm_model.fit(unmasked_sequence, y, epochs=5, 
              # callbacks=[generator_callback]
              )

Epoch 1/5
1/1 [==============================] - 1s 1s/step - loss: -0.8590
Epoch 2/5
1/1 [==============================] - 0s 11ms/step - loss: -1.3417
Epoch 3/5
1/1 [==============================] - 0s 11ms/step - loss: -1.7703
Epoch 4/5
1/1 [==============================] - 0s 9ms/step - loss: -2.1667
Epoch 5/5
1/1 [==============================] - 0s 12ms/step - loss: -2.4986


In [ ]:
masked_padded_sequence.shape

(4, 12)

In [ ]:



# total_input = keras.Input( (12,128),dtype=tf.int64)

# att_layer = layers.MultiHeadAttention(num_heads =8, key_dim = embedding_dimension)(x_input_vectors,
#                                                                                    x_input_vectors,
#                                                                                    x_input_vectors)
# attention_output = layers.Dropout(0.1)(att_layer)
# norm_layer = layers.LayerNormalization(epsilon = 1e-6)(attention_output+x_input_vectors)

# ff_net = tf.keras.Sequential([
#                               layers.Dense(128, activation= 'relu'),
#                               layers.Dense(embedding_dimension)

# ])
# ff_net_out = ff_net(norm_layer)
# ff_net_out  = layers.Dropout(0.1)(ff_net_out)

# sequence_output = layers.LayerNormalization(epsilon = 1e-6)(attention_output+ff_net_out)

# mlm_out = layers.Dense(vocab_size,  activation="softmax")(sequence_output)
# fin = layers.Dense(vocab_size)(mlm_out)
# mlm_model = keras.Model(inputs = total_input, outputs = sequence_output)
# mlm_model.compile(
#     loss = 'categorical_crossentrop',
#     optimizer = 'Adam',
#     metrics = ['accuracy']

# )
# mlm_model.fit(total_vectors, total_seqeunces)                 

# print(mlm_out.shape)

In [ ]:
sequence_output.shape


In [ ]:
input_layer1 = keras.Input(shape = (padding_length,128))
att_layer1 = layers.MultiHeadAttention(num_heads = 8, key_dim =embedding_dimension)(input_layer1,
                                                                         input_layer1,
                                                                         input_layer1)
drop_layer1 = layers.Dropout(0.1)(att_layer1)
norm_layer1 = layers.LayerNormalization(epsilon = 1e-6)(input_layer1 + drop_layer1)
ff1_layer1 = layers.Dense(embedding_dimension , activation='relu')(norm_layer1)
ff2_layer1 = layers.Dense(embedding_dimension)
drop2_layer1 = layers.Dropout(0.1)(ff1_layer1)
norm2_layer1 = layers.LayerNormalization(epsilon = 1e-6)(input_layer1 + ff2_layer1)
# output_layer1 = layers.Dense(128,activation = 'softmax')(att_layer1+ff2_layer1)
# small_model1 = keras.Model(input_layer1, output_layer1)

# small_model1.compile(
#     loss = 'categorical_crossentropy',
#     optimizer = 'Adam',
#     metrics = ['accuracy']
# )

# # small_model1.summary()
# small_model1.fit(total_vectors, total_sequences, epochs=100)

# ff2_layer1


In [ ]:
sample_token =  ['London is the at northern side of the [MASK]']


# tf_tokenizer= Tokenizer()

tokenized_sample , corpus = tokenizing_procedure(sample_token,tf_tokenizer)
print(tokenized_sample)
pad_sequences(tokenized_sample,maxlen = 10)
# masking(corpus, tokenized_sample,tf_tokenizer)

In [ ]:
prediction_sentence = 'my name is MASK'
tokenizing_procedure(prediction_sentence,tf_tokenizer)

In [ ]:


prediction_tokens, ikincil = tokenizing_procedure(prediction_sentence, tf_tokenizer)
np.argmax(small_model.predict(prediction_tokens))

In [ ]:
# input_layer = layers.Input(shape = (10,))
# l1_layer = layers.Embedding(corpus_length, 10, input_length =10)(input_layer)
# # l1_layer = layers.Flatten()(l1_layer)
# out_layer  = layers.Dense(10, activation= 'softmax')(l1_layer)
# tf_trial_model  = keras.Model(inputs= input_layer, outputs = out_layer)

# tf_trial_model.compile(
#     loss = 'categorical_crossentropy',
#     optimizer = 'Adam',
#     metrics = ['accuracy']
# )

# tf_trial_model.fit(
#                   #  np.transpose(masked_sequence),
#                    masked_sequence, 
#                    original_sequences, 
#                    epochs = 100)

In [ ]:
cukubi = ['my name is [MASK]',
          'I live in london']

vectorize_layer = TextVectorization(
        max_tokens=100,
        output_mode="int",
        # standardize=custom_standardization,
        output_sequence_length=10,
        
    )
sonc = vectorize_layer.adapt(cukubi)

In [ ]:
vectorize_layer.get_vocabulary()